In [ ]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cv2 as cv

# autoreload
%load_ext autoreload
%autoreload 2

Stereo to elevation map

In [ ]:
kitti_path = os.path.join('data', 'stereo_imgs', 'kitti', 'seq_00')

imgL = cv.imread(os.path.join(kitti_path, 'left_000000.png'), cv.IMREAD_GRAYSCALE)
imgR = cv.imread(os.path.join(kitti_path, 'right_000000.png'), cv.IMREAD_GRAYSCALE)
stereo = cv.StereoBM_create(numDisparities=16, blockSize=15)
disparity = stereo.compute(imgL,imgR)
plt.imshow(disparity,'gray')
plt.show()

In [ ]:
imgL = cv.imread('sol775_lmst154839_left.png', cv.IMREAD_GRAYSCALE)
imgR = cv.imread('sol775_lmst154839_right.png', cv.IMREAD_GRAYSCALE)
stereo = cv.StereoBM_create(numDisparities=16, blockSize=7)
disparity = stereo.compute(imgL,imgR)
plt.imshow(disparity,'gray')
plt.show()

In [ ]:
left = image.imread('sol775_lmst154839_left.png')
right = image.imread('sol775_lmst154839_right.png')

# Display the images
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 15))
ax1.imshow(left)
ax1.set_title('Left image')
ax2.imshow(right)
ax2.set_title('Right image')
plt.show()

Generate terrain map from sample image

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [ ]:
img = image.imread('data/jezero.png')
plt.imshow(img, origin='lower')

In [ ]:
img_gray = rgb2gray(img)
plt.imshow(img_gray, origin='lower', cmap='viridis_r')

In [ ]:
costmap = img_gray
m, n = costmap.shape

Local planner on costmap

In [ ]:
x = np.array([100, 100])  # current position

Compute optimal path from (0,0) to (781,926) as sequence of pixels through terrain map

Each pixel represents cost of traversing terrain at that location, with lower cost being better

In [ ]:
from astar import AStar

In [ ]:
class GlobalPlanner(AStar):

    def __init__(self, costmap):
        self.costmap = costmap
        self.width = costmap.shape[0]
        self.height = costmap.shape[1]

    def neighbors(self, node):
        x, y = node
        return [(nx, ny) for nx, ny in [(x+1, y), (x-1, y), (x, y+1), (x, y-1)] 
                if 0 <= nx < self.width and 0 <= ny < self.height]

    def distance_between(self, node1, node2):
        return self.costmap[node2]
    
    def heuristic_cost_estimate(self, node1, node2):
        """Straight line distance"""
        return np.linalg.norm(np.array(node1) - np.array(node2))

In [ ]:
gp_optimal = GlobalPlanner(costmap)

In [ ]:
optimal_path = list(gp_optimal.astar((0, 0), (m-1, n-1)))
dt = np.dtype('int32','int32')
optimal_path = np.array(optimal_path, dtype=dt)

In [ ]:
# Plot the path on the image
plt.rcParams["figure.figsize"] = [m/100, n/100]
plt.rcParams["figure.autolayout"] = True

fig, ax = plt.subplots()
im = ax.imshow(img_gray, extent=[0, n-1, 0, m-1], origin='lower', cmap='viridis_r')

# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.2)
plt.colorbar(im, cax=cax)

ax.plot(optimal_path[:,1], optimal_path[:,0], ls='dotted', linewidth=2, color='black')
plt.show()

In [ ]:
# Compute cost of optimal path
cost = 0
for node in optimal_path:
    cost += costmap[node[0], node[1]]
print(f'Cost of optimal path: {cost}')

Add noise to map to obtain imperfect "global" information

In [ ]:
sigma = 0.1
map_noisy = costmap + np.random.normal(0, sigma, size=(m,n))

In [ ]:
# Plot true map and noisy map
fig, ax = plt.subplots(1, 2, figsize=(2*m/100, n/100))
im1 = ax[0].imshow(img_gray, extent=[0, n-1, 0, m-1], origin='lower', cmap='viridis_r')
im2 = ax[1].imshow(map_noisy, extent=[0, n-1, 0, m-1], origin='lower', cmap='viridis_r')

In [ ]:
gp = GlobalPlanner(map_noisy)

In [ ]:
path = list(gp.astar((0, 0), (m-1, n-1)))
dt = np.dtype('int32','int32')
path = np.array(path, dtype=dt)

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(img_gray, extent=[0, n-1, 0, m-1], origin='lower', cmap='viridis_r')

# create an axes on the right side of ax. The width of cax will be 5%
# of ax and the padding between cax and ax will be fixed at 0.05 inch.
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.2)
plt.colorbar(im, cax=cax)

ax.plot(optimal_path[:,1], optimal_path[:,0], ls='dotted', linewidth=2, color='black')
ax.plot(path[:,1], path[:,0], ls='dotted', linewidth=2, color='red')
plt.show()

Add obstacle to map